In [2]:
!pip install praat-parselmouth
!pip install pyreaper

zsh:1: command not found: pip
zsh:1: command not found: pip


In [3]:
pip install praat-parselmouth

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 8.7 MB 510 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import librosa
import whisper
import pandas as pd
import parselmouth
from parselmouth.praat import call

# --- Existing Setup ---
model = whisper.load_model("base")
folder = "../WAVs"
features_list = []
long_pause_threshold = 3.0
chunk_size = 30  # seconds for pause distribution bins

# --- Feature Extraction Loop ---
for file in os.listdir(folder):
    if not file.endswith(".wav"):
        continue

    path = os.path.join(folder, file)
    print(f"Processing {file}...")

    try:
        # --- Existing Librosa and Whisper Processing ---
        y, sr = librosa.load(path, sr=16000)
        result = model.transcribe(path, word_timestamps=True)
        segments = result["segments"]
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        total_duration = librosa.get_duration(y=y, sr=sr)

        # --- Parselmouth Sound Object ---
        sound = parselmouth.Sound(path)

        # --- Advanced Feature Extraction using Parselmouth ---
        # Pitch
        pitch = sound.to_pitch()
        pitch_mean = call(pitch, "Get mean", 0, 0, "Hertz")
        pitch_std_dev = call(pitch, "Get standard deviation", 0, 0, "Hertz")

        # Intensity
        intensity = sound.to_intensity()
        intensity_mean = call(intensity, "Get mean", 0, 0, "dB")

        # Voice Quality Features
        point_process = call(sound, "To PointProcess (periodic, cc)", 75, 500)
        jitter_local = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        shimmer_local = call([sound, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        hnr = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_mean = call(hnr, "Get mean", 0, 0)

        # Formants
        formant = sound.to_formant_burg()
        f1_mean = call(formant, "Get mean", 1, 0, 0, "hertz")
        f2_mean = call(formant, "Get mean", 2, 0, 0, "hertz")

        # --- Existing Feature Calculations (with modifications) ---
        long_pause_durations = []
        speech_durations = []
        wpm_bins = []
        long_pause_distribution = [0] * int(np.ceil(total_duration / chunk_size))
        prev_end = 0

        for seg in segments:
            start, end = seg["start"], seg["end"]
            duration = end - start
            words = seg["text"].split()
            speech_durations.append(duration)

            pause_time = start - prev_end
            if pause_time > long_pause_threshold:
                long_pause_durations.append(pause_time)
                chunk_index = int(start // chunk_size)
                if chunk_index < len(long_pause_distribution):
                    long_pause_distribution[chunk_index] += pause_time

            prev_end = end
            if duration > 0:
                wpm = len(words) / (duration / 60)
                wpm_bins.append(wpm)

        total_long_pause = sum(long_pause_durations)
        avg_long_pause = np.mean(long_pause_durations) if long_pause_durations else 0
        long_pause_count = len(long_pause_durations)
        long_pause_ratio = total_long_pause / total_duration if total_duration > 0 else 0
        long_pause_std = np.std(long_pause_durations) if long_pause_durations else 0
        long_pause_jitter = np.mean(np.abs(np.diff(long_pause_durations))) if len(long_pause_durations) > 1 else 0

        total_speaking_time = sum(speech_durations)
        speaking_to_long_pause_ratio = total_speaking_time / total_long_pause if total_long_pause > 0 else float('inf')

        delta_wpm = np.diff(wpm_bins)
        acc_mean = np.mean(delta_wpm) if len(delta_wpm) > 1 else 0
        acc_std = np.std(delta_wpm) if len(delta_wpm) > 1 else 0

        speech_std = np.std(speech_durations)
        rhythm_outliers = sum(1 for d in speech_durations if abs(d - np.mean(speech_durations)) > 1.5 * speech_std)

        # --- Define Labels ---
        def label_long_pauses(pause_ratio, pause_count):
            if pause_ratio > 0.1 or pause_count > 5:
                return "Too Many Long Pauses"
            elif pause_ratio < 0.02:
                return "Very Few Long Pauses"
            else:
                return "Good Long Pause Control"

        def label_acceleration(acc_mean):
            if acc_mean > 5:
                return "Accelerating"
            elif acc_mean < -5:
                return "Decelerating"
            else:
                return "Steady Pace"

        def label_rhythm(rhythm_outliers):
            if rhythm_outliers > 3:
                return "Inconsistent Rhythm"
            elif rhythm_outliers == 0:
                return "Very Consistent"
            else:
                return "Consistent"

        label_pauses_val = label_long_pauses(long_pause_ratio, long_pause_count)
        label_accel_val = label_acceleration(acc_mean)
        label_rhythm_val = label_rhythm(rhythm_outliers)

        # --- Combine All Features ---
        combined_features = np.concatenate([
            mfcc_mean,
            [
                long_pause_count, total_long_pause, avg_long_pause, long_pause_ratio,
                acc_mean, acc_std, speech_std, rhythm_outliers,
                speaking_to_long_pause_ratio, long_pause_std, long_pause_jitter,
                pitch_mean, pitch_std_dev, intensity_mean, jitter_local, shimmer_local, hnr_mean,
                f1_mean, f2_mean
            ]
        ])

        features_list.append({
            "filename": file,
            "label_pauses": label_pauses_val,
            "label_acceleration": label_accel_val,
            "label_rhythm": label_rhythm_val,
            "features": combined_features,
            "long_pause_distribution": long_pause_distribution
        })

    except Exception as e:
        print(f"Could not process {file}: {e}")

# --- Save Enhanced Features ---
df = pd.DataFrame(features_list)
# Handle potential NaN or infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Fix: Only proceed if DataFrame is not empty
if not df.empty:
    feature_cols = [f"f{i}" for i in range(len(df.iloc[0]['features']))]
    features_expanded = pd.DataFrame(df["features"].tolist(), index=df.index)
    for col, name in enumerate(feature_cols):
        df[name] = features_expanded[col]
    df.drop(columns=["features"], inplace=True)
    df.to_csv("labeled_data_enhanced.csv", index=False)
    df.to_pickle("combined_features_enhanced.pkl")
    print("✅ Finished! Enhanced labeled data saved to 'labeled_data_enhanced.csv' and 'combined_features_enhanced.pkl'")
else:
    print("❌ No files were processed successfully. Check your audio files and dependencies.")


Processing ielts-listening-0092-cam14-test-3-part-4.wav...


/Users/Sammani/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processing short-stories-0207-209-sport-canada.wav...


/Users/Sammani/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processing short-stories-0264-267-etiquette.wav...


...

Processing AUDIO-2025-04-03-22-17-33.wav...


/Users/Sammani/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processing sandali_hci_audio.wav...


/Users/Sammani/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
!brew install ffmpeg

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
bstring: Fork of Paul Hsieh's Better String Library
domain-check: CLI tool for checking domain availability using RDAP and WHOIS protocols
lstr: Fast, minimalist directory tree viewer
mcp-proxy: Bridge between Streamable HTTP and stdio MCP transports
mcptools: CLI for interacting with MCP servers using both stdio and HTTP transport
pyrefly: Fast type checker and IDE for Python
rulesync: Unified AI rules management CLI tool

You have 4 outdated formulae installed.

==> Fetching downloads for: ffmpeg
==> Downloading https://ghcr.io/v2/homebrew/core/ffmpeg/manifests/7.1.1_3
######################################################################### 100.0%
==> Fetching dependencies for ffmpeg: aribb24, dav1d, 